В стандартной установке может отсутствовать часть модулей, необходимых для работы, их нужно установить  
Команда установки "pip install имя_модуля" вводится в этом же окне, но в формате кода.  
После установки необходимо перезапустить ядро выбрав в меню сверху Kernel->Restart.

In [1]:
import pandas as pd  # Для работы с таблицами данных (ДатаФреймы)
import requests  # Для запросов к серверу ИСС
import json  # Для обработки ответов сервера ИСС

# Выгрузка данных по акции Сбербанка
Сперва зададим изначально необходимые значения  
Сформируем базовый заспрос и пустой дата фрейм с именами колонок

In [2]:
# В базовом запросе мы укажем:
# engines/stok (фондовый рынок), markets/shares - рынок акций, boards/TQBR - режим торгов для акции сбербанка TQBR
# Базовый запрос поместим в переменную base_url
base_url = "http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities.json"
response = requests.get(base_url)  # Получим ответ от сервера
result = json.loads(response.text)  # Расшифровываем ответ от сервера
# Задаем имена колонок извлекая данные из ответа сервера
col_name = result['history']['columns']
data_shares = pd.DataFrame(columns=col_name)

In [3]:
data_shares  # Можно посмотреть что получилось

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR


Получился датафрйем из 23 колонок  
Перейдем к получению данных по акциям Сбербанка

In [5]:
# Первая страница данных
url_share = 'http://iss.moex.com/iss/history/engines/stock/markets/shares/boards/TQBR/securities/SBER.json'
# Получаем данные
response = requests.get(url_share)
result = json.loads(response.text)
resp_date = result['history']['data']
# Извлеченные данные добавляем в начало дата фрейма
data_shares = pd.DataFrame(resp_date, columns=col_name)
# Узнаем количество полученных строк, если их меньше 100, то мы получили все данные
a = len(resp_date)

Если же количество строк 100, то получены не все данные (сервер отправляет ограниченное количество строк за раз).  
В этом случае запускаем в цикле переборку остальных страниц.  
Цикл будет работать до тех пор, пока мы не получим количество строк меньше, чем 100.  
Для этого каждый раз на предпоследнем шаге цикла переопределяем переменную.

In [6]:
b = 100
while a == 100:
    url_opt = '?start=' + str(b)
    url_next_page  = url_share + url_opt
    response = requests.get(url_next_page)
    result = json.loads(response.text)
    resp_date = result['history']['data']
    data_next_page = pd.DataFrame(resp_date, columns = col_name)
    data_shares = pd.concat([data_shares, data_next_page], ignore_index=True) 
    a = len(resp_date)
    b = b + 100

Посмотрим сколько примерно строк получилось, узнав значение b

In [7]:
b

2700

За короткое время было загружено более 2600 строк данных  
Точное колисество строк можно узнать отобразил наш датафрейм

In [8]:
data_shares

,BOARDID,TRADEDATE,SHORTNAME,SECID,NUMTRADES,VALUE,OPEN,LOW,HIGH,LEGALCLOSEPRICE,...,MARKETPRICE2,MARKETPRICE3,ADMITTEDQUOTE,MP2VALTRD,MARKETPRICE3TRADESVALUE,ADMITTEDVALUE,WAVAL,TRADINGSESSION,CURRENCYID,TRENDCLSPR
0,TQBR,2013-03-25,Сбербанк,SBER,140,5.934000e+07,96.00,96.00,101.14,98.66,...,99.99,99.99,99.99,8.572434e+09,8.572434e+09,8572433621.95,NaN,3,SUR,NaN
1,TQBR,2013-03-26,Сбербанк,SBER,183,1.260304e+08,98.58,97.08,99.31,97.13,...,97.82,97.82,97.82,1.221847e+10,1.221847e+10,12218468579.299999,NaN,3,SUR,-1.61
2,TQBR,2013-03-27,Сбербанк,SBER,176,1.218359e+08,97.90,95.39,98.00,96.73,...,96.45,96.45,96.45,1.251981e+10,1.251981e+10,12519812986.549999,NaN,3,SUR,-0.46
3,TQBR,2013-03-28,Сбербанк,SBER,299,1.924698e+08,96.38,95.72,98.66,98.64,...,97.74,97.74,97.74,1.257281e+10,1.257281e+10,12572809760.42,NaN,3,SUR,1.90
4,TQBR,2013-03-29,Сбербанк,SBER,121,7.726886e+07,98.60,98.32,99.09,98.83,...,98.77,98.77,98.77,2.611474e+09,2.611474e+09,2611473832.5,NaN,3,SUR,0.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2654,TQBR,2023-10-03,Сбербанк,SBER,100698,6.193849e+09,258.99,257.23,260.50,259.89,...,258.93,258.93,None,5.478216e+09,5.478216e+09,None,0.0,3,SUR,0.26
2655,TQBR,2023-10-04,Сбербанк,SBER,56512,4.816207e+09,259.00,258.24,260.39,259.68,...,259.28,259.28,None,3.879693e+09,3.879693e+09,None,0.0,3,SUR,-0.05
2656,TQBR,2023-10-05,Сбербанк,SBER,76868,6.111705e+09,260.70,258.26,261.39,259.05,...,260.22,260.22,None,5.274428e+09,5.274428e+09,None,0.0,3,SUR,-0.06
2657,TQBR,2023-10-06,Сбербанк,SBER,83397,8.338547e+09,259.77,257.86,263.40,260.42,...,259.10,259.10,None,4.423401e+09,4.423401e+09,None,0.0,3,SUR,1.37
